In [47]:
import sys
sys.path.append('/usr/local/lib/python2.7/site-packages')

import os
import cv2
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation, Dropout, Flatten, Dense, Lambda
from keras.layers import ELU
from keras.optimizers import Adam
import keras.backend.tensorflow_backend as KTF


N_img_height = 66
N_img_width = 220
N_img_channels = 3
def nvidia_model():
    inputShape = (N_img_height, N_img_width, N_img_channels)

    model = Sequential()
    # normalization    
    # perform custom normalization before lambda layer in network
    model.add(Lambda(lambda x: x/ 127.5 - 1, input_shape = inputShape))

    model.add(Convolution2D(24, (5, 5), 
                            strides=(2,2), 
                            padding = 'valid',
                            kernel_initializer = 'he_normal',
                            name = 'conv1'))
    
    
    model.add(ELU())    
    model.add(Convolution2D(36, (5, 5), 
                            strides=(2,2), 
                            padding = 'valid',
                            kernel_initializer = 'he_normal',
                            name = 'conv2'))
    
    model.add(ELU())    
    model.add(Convolution2D(48, (5, 5), 
                            strides=(2,2), 
                            padding = 'valid',
                            kernel_initializer = 'he_normal',
                            name = 'conv3'))
    model.add(ELU())
    model.add(Dropout(0.5))
    model.add(Convolution2D(64, (3, 3), 
                            strides = (1,1), 
                            padding = 'valid',
                            kernel_initializer = 'he_normal',
                            name = 'conv4'))
    
    model.add(ELU())              
    model.add(Convolution2D(64, (3, 3), 
                            strides= (1,1), 
                            padding = 'valid',
                            kernel_initializer = 'he_normal',
                            name = 'conv5'))
              
              
    model.add(Flatten(name = 'flatten'))
    model.add(ELU())
    model.add(Dense(100, kernel_initializer = 'he_normal', name = 'fc1'))
    model.add(ELU())
    model.add(Dense(50, kernel_initializer = 'he_normal', name = 'fc2'))
    model.add(ELU())
    model.add(Dense(10, kernel_initializer = 'he_normal', name = 'fc3'))
    model.add(ELU())
    
    # do not put activation at the end because we want to exact output, not a class identifier
    model.add(Dense(1, name = 'output', kernel_initializer = 'he_normal'))
    
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer = adam, loss = 'mse')

    return model

In [26]:
model = nvidia_model()
model.load_weights('model-weights-Vtest3.h5')

In [89]:
def opticalFlowOverlay(image_current, image_next):
    """
    input: image_currentdef preprocess_image_from_path(image_path, scale_factor=0.5, bright_factor=1):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = change_brightness(img, bright_factor)
    img = crop_image(img, scale_factor)
    return img, image_next (RGB images)
    output: mask
    """
    feature_params = dict( maxCorners = 500,
                       qualityLevel = 0.1,
                       minDistance = 7,
                       blockSize = 5 )
    lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
    image_current_saved = np.copy(image_current)
    image_next_saved = np.copy(image_next)
    
    image_current = cv2.cvtColor(image_current, cv2.COLOR_RGB2GRAY)
    image_next = cv2.cvtColor(image_next, cv2.COLOR_RGB2GRAY)
    
    # Finds edges in an image using the [Canny86] algorithm.
    p0 = cv2.goodFeaturesToTrack(image_current, mask = None, **feature_params)

    p1, st, err = cv2.calcOpticalFlowPyrLK(image_current, image_next, p0, None, **lk_params)


    color = np.random.randint(0, 255, (100, 3))

    mask = np.zeros_like(image_current)

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel() # flatten
        c, d = old.ravel()
        mask = cv2.arrowedLine(mask, (a,b), (c, d), color[i%100].tolist(), 2, 8)
        
        image_next = cv2.circle(image_next_saved, (a, b), 1, color[i%100].tolist(), -1)
        image_next_fg = cv2.bitwise_and(image_next, image_next, mask = mask)
        
    dst = cv2.add(image_next, image_next_fg)
    return dst

In [28]:
def crop_image(image, scale):
    """
    preprocesses the image
    
    input: image (480 (y), 640 (x), 3) RGB
    output: image (shape is (66, 220, 3) as RGB)
    
    This stuff is performed on my validation data and my training data
    Process: 
             1) Cropping out black spots
             3) resize to (66, 220, 3) if not done so already from perspective transform
    """
    # Crop out sky (top 130px) and the hood of the car (bottom 270px) 
    image_cropped = image[130:370,:] # -> (240, 640, 3)
    
    height = int(240*scale)
    width = int(640*scale)
    image = cv2.resize(image_cropped, (220, 66), interpolation = cv2.INTER_AREA)
    
    return image

In [29]:
def opticalFlowDense(image_current, image_next):
    """
    input: image_current, image_next (RGB images)
    calculates optical flow magnitude and angle and places it into HSV image
    * Set the saturation to the saturation value of image_next
    * Set the hue to the angles returned from computing the flow params
    * set the value to the magnitude returned from computing the flow params
    * Convert from HSV to RGB and return RGB image with same size as original image
    """
    gray_current = cv2.cvtColor(image_current, cv2.COLOR_RGB2GRAY)
    gray_next = cv2.cvtColor(image_next, cv2.COLOR_RGB2GRAY)
    
    hsv = np.zeros(image_current.shape)
    # set saturation
    hsv[:,:,1] = cv2.cvtColor(image_next, cv2.COLOR_RGB2HSV)[:,:,1]
 
    # Flow Parameters
    flow_mat = None
    image_scale = 0.5
    nb_images = 1
    win_size = 15
    nb_iterations = 2
    deg_expansion = 5
    STD = 1.3
    extra = 0

    # obtain dense optical flow paramters
    flow = cv2.calcOpticalFlowFarneback(gray_current, gray_next,  
                                        flow_mat, 
                                        image_scale, 
                                        nb_images, 
                                        win_size, 
                                        nb_iterations, 
                                        deg_expansion, 
                                        STD, 
                                        0)
                                        
        
    # convert from cartesian to polar
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])  
        
    # hue corresponds to direction
    hsv[:,:,0] = ang * (180/ np.pi / 2)
    
    # value corresponds to magnitude
    hsv[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # convert HSV to float32's
    hsv = np.asarray(hsv, dtype= np.float32)    

    rgb_flow = cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)

    
    return rgb_flow

In [37]:
def preprocess_image_from_path(image_path, scale_factor=0.5, bright_factor=1):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = crop_image(img, scale_factor)
    return img

In [38]:
ls data

driving.csv      predict.csv      test_frames/     train.txt
predict/         test.mp4         train.mp4        training_frames/


In [39]:
def capture_frames(video_source, speed_data):
    
    '''
    Captures .mp4 video frames to .jpg images and creates a .csv to store the capture information
    '''
    
    num_frames = speed_data.shape[0]
    
    # create VideoCapture instance
    cap = cv2.VideoCapture(video_source)
    # set frame count
    cap.set(cv2.CAP_PROP_FRAME_COUNT, num_frames)
    
    with open('./data/predict.csv', 'w') as csvfile:
        fieldnames = ['image_path', 'frame', 'speed']
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
        writer.writeheader()

        for idx in xrange(num_frames):
            # set frame index
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            # read the frame
            success, image = cap.read()

            if success:
                image_path = os.path.join('./data/test_frames/', str(idx) + '.jpg')

                # save image to IMG folder
                cv2.imwrite(image_path, image)

                # write row to driving.csv
                writer.writerow({'image_path': image_path,
                         'frame': idx,
                         'speed': speed_data[idx],
                        })
            else:
                print 'Failed to read frame ', idx
        
        print 'Done!'

In [40]:
data = pd.read_csv('./data/driving.csv')

In [41]:
data.head(10)

,image_path,frame,speed
0,./data/training_frames/0.jpg,0,28.105569
1,./data/training_frames/1.jpg,1,28.105569
2,./data/training_frames/2.jpg,2,28.106527
3,./data/training_frames/3.jpg,3,28.130404
4,./data/training_frames/4.jpg,4,28.109243
5,./data/training_frames/5.jpg,5,28.088572
6,./data/training_frames/6.jpg,6,28.034211
7,./data/training_frames/7.jpg,7,28.018491
8,./data/training_frames/8.jpg,8,27.986624
9,./data/training_frames/9.jpg,9,28.016352


In [42]:
len(data.index)

20400

In [90]:
for i in range(len(data)-1):
    y1 = data.iloc[i]['speed']
    y2 = data.iloc[i+1]['speed']
    
    x1 = preprocess_image_from_path(data.iloc[i]['image_path'])
    x2 = preprocess_image_from_path(data.iloc[i+1]['image_path'])
    
    img1 = cv2.cvtColor(cv2.imread(data.iloc[i]['image_path']), cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(cv2.imread(data.iloc[i+1]['image_path']), cv2.COLOR_BGR2RGB)
        
    rgb_diff = opticalFlowDense(x1, x2)
    rgb_diff = rgb_diff.reshape(1, rgb_diff.shape[0], rgb_diff.shape[1], rgb_diff.shape[2])
    avg_speed = np.array([[np.mean([y1,y2])]])
    
    prediction = model.predict(rgb_diff)
    error = abs(prediction - y2)
    truth = y2
    
    predict_path = os.path.join('./data/predict/', str(i) + '.jpg')
    
    dst = np.copy(img2)
    
    dst = opticalFlowOverlay(img1, img2) # This is a sparse optical flow overlay
    
    # to write new image via openCV
    offset = 50
    FONT_SIZE = 1
    THICKNESS = 2
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(dst,'pred: ' + str(prediction[0][0])[:5],(10,offset), font, FONT_SIZE,(255,250,40), THICKNESS,cv2.LINE_AA)
    cv2.putText(dst,'truth: ' + str(y2)[:5],(10,offset+50), font, FONT_SIZE,(35,255,70), THICKNESS,cv2.LINE_AA)
    cv2.putText(dst, 'error: ' + str(error[0][0])[:5], (10, offset+100),font, FONT_SIZE, (255, 120, 80), THICKNESS, cv2.LINE_AA)
    
    # convert back to BGR for writing
    dst = cv2.cvtColor(dst, cv2.COLOR_RGB2BGR)
    cv2.imwrite(predict_path, dst)
    
print('done!')
    

done!


In [97]:
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip
import glob
import os

images = ['./data/predict/' + str(i+1) + '.jpg' for i in range(0, 20398)]
clip = ImageSequenceClip(images, fps=60)
clip.write_videofile("predicted_speed_fast.mp4", fps=60)
print('done!')

[MoviePy] >>>> Building video predicted_speed_fast.mp4
[MoviePy] Writing video predicted_speed_fast.mp4






  0%|          | 0/20399 [00:00<?, ?it/s]



  0%|          | 7/20399 [00:00<05:04, 66.93it/s]



  0%|          | 14/20399 [00:00<05:07, 66.27it/s]



  0%|          | 22/20399 [00:00<05:03, 67.21it/s]



  0%|          | 29/20399 [00:00<05:03, 67.19it/s]



  0%|          | 37/20399 [00:00<04:54, 69.21it/s]



  0%|          | 44/20399 [00:00<05:12, 65.14it/s]



  0%|          | 51/20399 [00:00<05:23, 62.89it/s]



  0%|          | 58/20399 [00:00<05:26, 62.25it/s]



  0%|          | 64/20399 [00:00<05:32, 61.23it/s]



  0%|          | 71/20399 [00:01<05:27, 62.06it/s]



  0%|          | 78/20399 [00:01<05:28, 61.90it/s]



  0%|          | 85/20399 [00:01<05:24, 62.57it/s]



  0%|          | 92/20399 [00:01<05:20, 63.29it/s]



  0%|          | 99/20399 [00:01<05:21, 63.09it/s]



  1%|          | 106/20399 [00:01<05:37, 60.06it/s]



  1%|          | 113/20399 [00:01<05:39, 59.83it/s]



  1%|          | 119/20399 [00:01<05:45, 58.71it/s]



  1%|          | 125/20399 [00:

  4%|▍         | 794/20399 [00:13<06:01, 54.18it/s]



  4%|▍         | 800/20399 [00:13<05:51, 55.70it/s]



  4%|▍         | 806/20399 [00:13<05:53, 55.43it/s]



  4%|▍         | 812/20399 [00:13<05:46, 56.47it/s]



  4%|▍         | 818/20399 [00:13<05:48, 56.18it/s]



  4%|▍         | 824/20399 [00:14<05:44, 56.80it/s]



  4%|▍         | 830/20399 [00:14<05:46, 56.45it/s]



  4%|▍         | 836/20399 [00:14<05:43, 56.88it/s]



  4%|▍         | 842/20399 [00:14<05:41, 57.34it/s]



  4%|▍         | 848/20399 [00:14<05:39, 57.57it/s]



  4%|▍         | 854/20399 [00:14<05:37, 57.94it/s]



  4%|▍         | 860/20399 [00:14<05:38, 57.80it/s]



  4%|▍         | 866/20399 [00:14<05:45, 56.58it/s]



  4%|▍         | 872/20399 [00:14<05:49, 55.95it/s]



  4%|▍         | 878/20399 [00:15<05:54, 55.14it/s]



  4%|▍         | 884/20399 [00:15<06:00, 54.14it/s]



  4%|▍         | 890/20399 [00:15<06:04, 53.49it/s]



  4%|▍         | 896/20399 [00:15<06:07, 53.03it/s]



  4%|▍    

  7%|▋         | 1494/20399 [00:27<05:52, 53.63it/s]



  7%|▋         | 1500/20399 [00:27<05:59, 52.53it/s]



  7%|▋         | 1506/20399 [00:27<06:00, 52.40it/s]



  7%|▋         | 1512/20399 [00:27<06:05, 51.70it/s]



  7%|▋         | 1518/20399 [00:27<06:07, 51.31it/s]



  7%|▋         | 1524/20399 [00:27<06:04, 51.78it/s]



  8%|▊         | 1530/20399 [00:28<05:58, 52.66it/s]



  8%|▊         | 1536/20399 [00:28<05:57, 52.71it/s]



  8%|▊         | 1542/20399 [00:28<05:51, 53.68it/s]



  8%|▊         | 1548/20399 [00:28<05:54, 53.22it/s]



  8%|▊         | 1554/20399 [00:28<05:56, 52.90it/s]



  8%|▊         | 1560/20399 [00:28<05:58, 52.55it/s]



  8%|▊         | 1566/20399 [00:28<06:02, 51.91it/s]



  8%|▊         | 1572/20399 [00:28<06:01, 52.06it/s]



  8%|▊         | 1578/20399 [00:29<05:56, 52.77it/s]



  8%|▊         | 1584/20399 [00:29<05:51, 53.58it/s]



  8%|▊         | 1590/20399 [00:29<05:49, 53.84it/s]



  8%|▊         | 1596/20399 [00:29<05:44, 54.62i

 11%|█         | 2214/20399 [00:41<06:04, 49.85it/s]



 11%|█         | 2220/20399 [00:41<06:10, 49.01it/s]



 11%|█         | 2226/20399 [00:41<06:00, 50.46it/s]



 11%|█         | 2232/20399 [00:41<05:46, 52.51it/s]



 11%|█         | 2238/20399 [00:41<05:40, 53.32it/s]



 11%|█         | 2244/20399 [00:41<05:38, 53.56it/s]



 11%|█         | 2250/20399 [00:41<05:39, 53.44it/s]



 11%|█         | 2256/20399 [00:41<05:57, 50.74it/s]



 11%|█         | 2262/20399 [00:41<05:59, 50.50it/s]



 11%|█         | 2268/20399 [00:42<06:03, 49.82it/s]



 11%|█         | 2274/20399 [00:42<06:55, 43.59it/s]



 11%|█         | 2279/20399 [00:42<06:46, 44.56it/s]



 11%|█         | 2284/20399 [00:42<06:49, 44.24it/s]



 11%|█         | 2289/20399 [00:42<06:57, 43.41it/s]



 11%|█         | 2294/20399 [00:42<06:41, 45.06it/s]



 11%|█▏        | 2300/20399 [00:42<06:15, 48.16it/s]



 11%|█▏        | 2306/20399 [00:42<06:09, 49.02it/s]



 11%|█▏        | 2311/20399 [00:43<06:06, 49.30i

...

 89%|████████▊ | 18090/20399 [05:28<00:40, 57.24it/s]



 89%|████████▊ | 18096/20399 [05:28<00:40, 56.55it/s]



 89%|████████▊ | 18102/20399 [05:28<00:40, 56.43it/s]



 89%|████████▉ | 18108/20399 [05:28<00:41, 55.75it/s]



 89%|████████▉ | 18114/20399 [05:28<00:40, 55.81it/s]



 89%|████████▉ | 18120/20399 [05:28<00:40, 55.84it/s]



 89%|████████▉ | 18126/20399 [05:28<00:40, 55.74it/s]



 89%|████████▉ | 18132/20399 [05:29<00:40, 55.88it/s]



 89%|████████▉ | 18138/20399 [05:29<00:40, 56.02it/s]



 89%|████████▉ | 18144/20399 [05:29<00:39, 56.46it/s]



 89%|████████▉ | 18150/20399 [05:29<00:39, 56.99it/s]



 89%|████████▉ | 18156/20399 [05:29<00:39, 56.64it/s]



 89%|████████▉ | 18162/20399 [05:29<00:39, 56.27it/s]



 89%|████████▉ | 18168/20399 [05:29<00:39, 55.98it/s]



 89%|████████▉ | 18174/20399 [05:29<00:39, 56.41it/s]



 89%|████████▉ | 18180/20399 [05:29<00:38, 57.00it/s]



 89%|████████▉ | 18186/20399 [05:29<00:38, 57.49it/s]



 89%|████████▉ | 18192/20399 [0

 92%|█████████▏| 18798/20399 [05:41<00:30, 53.32it/s]



 92%|█████████▏| 18804/20399 [05:41<00:29, 53.58it/s]



 92%|█████████▏| 18810/20399 [05:41<00:29, 53.60it/s]



 92%|█████████▏| 18816/20399 [05:41<00:29, 53.51it/s]



 92%|█████████▏| 18822/20399 [05:41<00:29, 53.26it/s]



 92%|█████████▏| 18828/20399 [05:41<00:29, 53.55it/s]



 92%|█████████▏| 18834/20399 [05:41<00:29, 53.74it/s]



 92%|█████████▏| 18840/20399 [05:41<00:29, 53.48it/s]



 92%|█████████▏| 18846/20399 [05:41<00:29, 53.53it/s]



 92%|█████████▏| 18852/20399 [05:42<00:28, 53.87it/s]



 92%|█████████▏| 18858/20399 [05:42<00:28, 53.94it/s]



 92%|█████████▏| 18864/20399 [05:42<00:28, 53.78it/s]



 93%|█████████▎| 18870/20399 [05:42<00:28, 53.53it/s]



 93%|█████████▎| 18876/20399 [05:42<00:28, 53.82it/s]



 93%|█████████▎| 18882/20399 [05:42<00:27, 54.20it/s]



 93%|█████████▎| 18888/20399 [05:42<00:27, 54.20it/s]



 93%|█████████▎| 18894/20399 [05:42<00:27, 54.29it/s]



 93%|█████████▎| 18900/20399 [0

 96%|█████████▌| 19507/20399 [05:53<00:15, 57.28it/s]



 96%|█████████▌| 19513/20399 [05:53<00:15, 57.46it/s]



 96%|█████████▌| 19519/20399 [05:53<00:15, 57.67it/s]



 96%|█████████▌| 19525/20399 [05:54<00:15, 57.63it/s]



 96%|█████████▌| 19531/20399 [05:54<00:15, 57.56it/s]



 96%|█████████▌| 19537/20399 [05:54<00:14, 57.64it/s]



 96%|█████████▌| 19543/20399 [05:54<00:14, 57.47it/s]



 96%|█████████▌| 19549/20399 [05:54<00:14, 57.65it/s]



 96%|█████████▌| 19555/20399 [05:54<00:14, 56.96it/s]



 96%|█████████▌| 19561/20399 [05:54<00:14, 57.06it/s]



 96%|█████████▌| 19567/20399 [05:54<00:14, 56.25it/s]



 96%|█████████▌| 19573/20399 [05:54<00:14, 56.73it/s]



 96%|█████████▌| 19579/20399 [05:54<00:14, 56.83it/s]



 96%|█████████▌| 19585/20399 [05:55<00:14, 57.25it/s]



 96%|█████████▌| 19591/20399 [05:55<00:14, 57.44it/s]



 96%|█████████▌| 19597/20399 [05:55<00:13, 57.38it/s]



 96%|█████████▌| 19603/20399 [05:55<00:14, 56.85it/s]



 96%|█████████▌| 19609/20399 [0

 99%|█████████▉| 20217/20399 [06:06<00:03, 56.90it/s]



 99%|█████████▉| 20223/20399 [06:06<00:03, 57.01it/s]



 99%|█████████▉| 20229/20399 [06:06<00:02, 57.66it/s]



 99%|█████████▉| 20235/20399 [06:06<00:02, 57.59it/s]



 99%|█████████▉| 20241/20399 [06:06<00:02, 57.44it/s]



 99%|█████████▉| 20247/20399 [06:06<00:02, 56.97it/s]



 99%|█████████▉| 20253/20399 [06:06<00:02, 57.09it/s]



 99%|█████████▉| 20259/20399 [06:06<00:02, 56.96it/s]



 99%|█████████▉| 20265/20399 [06:07<00:02, 56.53it/s]



 99%|█████████▉| 20271/20399 [06:07<00:02, 56.95it/s]



 99%|█████████▉| 20277/20399 [06:07<00:02, 57.07it/s]



 99%|█████████▉| 20283/20399 [06:07<00:02, 57.43it/s]



 99%|█████████▉| 20289/20399 [06:07<00:01, 57.05it/s]



 99%|█████████▉| 20295/20399 [06:07<00:01, 56.29it/s]



100%|█████████▉| 20301/20399 [06:07<00:01, 56.58it/s]



100%|█████████▉| 20307/20399 [06:07<00:01, 56.99it/s]



100%|█████████▉| 20313/20399 [06:07<00:01, 56.82it/s]



100%|█████████▉| 20319/20399 [0

[MoviePy] Done.
[MoviePy] >>>> Video ready: predicted_speed_fast.mp4 

done!
